In [1]:
import lxml.html as parser
import requests
import csv
import re
from urllib.parse import urlsplit, urljoin
import time
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

In [6]:
def get_float(string_):
    return float(''.join(re.findall('\d+',string_)))
def get_cep(string_):
    return ''.join(re.findall('\d{5}-\d{3}',string_))
def sub_string(string_):
    return string_.strip()
def get_links_page(start_url):
    r = requests.get(start_url)
    html = parser.fromstring(r.text)
    links = html.xpath("//a[@class='OLXad-list-link']/@href")
    links = [urljoin(start_url, l) for l in links]
    links = [urlsplit(l)._replace(query="").geturl() for l in links]
    next_page = html.xpath("//div[@class='module_pagination']//ul[@class='list']//li[@class='item number']/a/@href")[0]
    return links, next_page
def get_atributes(links):
    tp_imovel = []
    condominio = []
    iptu = []
    area = []
    quartos = []
    banheiros = []
    garagem = []
    prices = []
    cep = []
    bairro = []
    city = []
    url = links
    for i in links:
        time.sleep(random.randint(1,3))
        r = requests.get(i)
        product_html = parser.fromstring(r.text)        
        des_dict = dict(zip(
            product_html.xpath("//ul[@class='list square-gray']/li[@class='item']/p[@class='text']/span[@class='term']/text()"),
            product_html.xpath("//ul[@class='list square-gray']/li[@class='item']/p[@class='text']/strong[@class='description']/text()")))

        try:
            tp_imovel.append(sub_string(des_dict['Tipo:']))
        except:
            tp_imovel.append('')
        try:
            condominio.append(get_float(des_dict['Condomínio:']))
        except:
            condominio.append(None)
        try:
            iptu.append(get_float(des_dict['IPTU:']))
        except:
            iptu.append(None)
        try:
            area.append(get_float(des_dict['Área útil:']))
        except:
            area.append(None)
        try:
            quartos.append(get_float(des_dict['Número de quartos:']))
        except:
            quartos.append(None)
        try:
            banheiros.append(get_float(des_dict['Número de banheiros:']))
        except:
            banheiros.append(None)
        try:
            garagem.append(get_float(des_dict['Vagas na garagem:']))
        except:
            garagem.append(None)
        try:
            city.append(sub_string(des_dict['Município:']))
        except:
            city.append('')
        try:
            cep.append(get_cep(des_dict['CEP do imóvel:']))
        except:
            cep.append('')
        try:
            bairro.append(sub_string(des_dict['Bairro:']))
        except:
            bairro.append('')
        try:
            prices.append(get_float(product_html.xpath("//h3[@class='price']/span[@class='actual-price']/text()")[0]))
        except:
            prices.append(None)
    df = pd.DataFrame({'tp_imovel':tp_imovel,'price':prices,'cond':condominio,
                       'iptu':iptu,'area':area,'quartos':quartos,'banheiros':banheiros,
                       'garagem':garagem,'cep':cep,'City':city,'Bairro':bairro,'url':links})        
    return df

In [ ]:
df_t = pd.DataFrame()
next_page = 'https://sp.olx.com.br/grande-campinas/regiao-de-campinas/campinas/imoveis/aluguel/2-quartos?gsp=1'
for i in range(150):
    print('Página =>',i)
    try:
        n_links, next_page = get_links_page(next_page)
        df = get_atributes(n_links)
        df_t = pd.concat([df_t,df])
    except:
        print('Erro na busca de novas páginas.')
        break
df_t.to_excel('Anúncios_28_01_2019.xlsx')

Página => 0
